## Overview результатов и интересные инсайды

1. Большая часть пользователей не из Московского региона
2. Больше трети пользователей незарегистрировано ни в одной из компаний
3. Чуть больше половины пользователей зарегистрировано только в одной компании, а не в нескольких
4. Пользователь может иметь аккаунт в нескольких компаниях
5. Большая часть пользователей из Red Company
6. Если сравнивать Blue, Green и Black, то у Blue больше всего пользователей, но они приносят меньше всего GVM
7. в Green и Black есть прослойка пользователей, которая не пользуется ни одним из девайсов, но всё также приносит GVM
8. Ни местонахождение (Москва или Регионы), ни тип девайса не влияют на GVM

## Импорт данных

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
df = pd.read_excel('test_file.xlsx', header=1, index_col=0)

## Небольшое Data Quality

Проверка на NaN values

In [3]:
df.isna().any()

hid                      False
active_Red               False
has_Green_id             False
has_Black_id             False
has_Blue_id              False
Blue_activity_count      False
Blue_gmv_total            True
Green_gvm_total          False
Green_gvm_order_count    False
Black_gvm_total          False
Black_gvm_order_count    False
Red_android              False
Green_android            False
Black_android            False
Blue_android             False
Red_apple                False
Green_apple              False
Black_apple              False
Blue_apple               False
is_msk                   False
Red_web                  False
Green_web                False
Black_web                False
Blue_web                 False
dtype: bool

In [4]:
df[df['Blue_gmv_total'].isna()]

,hid,active_Red,has_Green_id,has_Black_id,has_Blue_id,Blue_activity_count,Blue_gmv_total,Green_gvm_total,Green_gvm_order_count,Black_gvm_total,...,Blue_android,Red_apple,Green_apple,Black_apple,Blue_apple,is_msk,Red_web,Green_web,Black_web,Blue_web
68383,18f44eb8c1abb256416294171fb63cba,0.0,1.0,0.0,3.0,4.0,NaN,820.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Так как кейс всего один и то он относится к косячному кейсу, который я описал ниже, заменять никак NaN я не буду, строчка просто удалиться далее

Судя по документации, принадлежность пользователя к компании, городу (is_msk) и девайсу (Яблоко, АндроЕд, десктопная версия) это бинарное значение, которое может принимать 0, либо 1.
<br> Необходимо проверить так ли это на самом деле и косячные кейсы исключить

In [5]:
columns_to_check = ['active_Red', 
                    'has_Green_id', 
                    'has_Black_id', 
                    'has_Blue_id', 
                    'Red_android', 
                    'Green_android',
                    'Black_android',
                    'Blue_android',
                    'Red_apple',
                    'Green_apple',
                    'Black_apple',
                    'Blue_apple',
                    'is_msk',
                    'Red_web',
                    'Green_web',
                    'Black_web',
                    'Blue_web']

In [6]:
clean_df = df[~(df[columns_to_check] > 1).any(axis=1)]

Также в некоторых числовых по смыслу колонках в исходном датасете были замечены datetime объекты, что я также буду считать ошибочными значениями.
<br> Так как некоторые такие колонки имеют тип object я поменяю их тип на float64 как и остальные, а в тех случаях, где выдаст ошибку - исправлю

In [7]:
clean_df.dtypes

hid                       object
active_Red               float64
has_Green_id             float64
has_Black_id             float64
has_Blue_id              float64
Blue_activity_count       object
Blue_gmv_total           float64
Green_gvm_total          float64
Green_gvm_order_count     object
Black_gvm_total          float64
Black_gvm_order_count     object
Red_android              float64
Green_android            float64
Black_android            float64
Blue_android             float64
Red_apple                float64
Green_apple              float64
Black_apple              float64
Blue_apple               float64
is_msk                   float64
Red_web                  float64
Green_web                float64
Black_web                float64
Blue_web                 float64
dtype: object

In [8]:
columns_to_convert = ['Blue_activity_count', 'Green_gvm_order_count', 'Black_gvm_order_count']

In [9]:
clean_df[columns_to_convert] = clean_df[columns_to_convert].astype('float64')

C:\Users\60112813\AppData\Local\Temp\ipykernel_22120\567450683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[columns_to_convert] = clean_df[columns_to_convert].astype('float64')


In [10]:
clean_df.dtypes

hid                       object
active_Red               float64
has_Green_id             float64
has_Black_id             float64
has_Blue_id              float64
Blue_activity_count      float64
Blue_gmv_total           float64
Green_gvm_total          float64
Green_gvm_order_count    float64
Black_gvm_total          float64
Black_gvm_order_count    float64
Red_android              float64
Green_android            float64
Black_android            float64
Blue_android             float64
Red_apple                float64
Green_apple              float64
Black_apple              float64
Blue_apple               float64
is_msk                   float64
Red_web                  float64
Green_web                float64
Black_web                float64
Blue_web                 float64
dtype: object

Ошибок никаких не было при конвертации, все колонки имеют нужный тип данных по логике, можно переходить к EDA

## EDA Анализ

### Кол-во пользователей

In [11]:
clean_df['hid'].nunique()

99475

In [12]:
clean_df['hid'].duplicated().any()

False

Всего у нас 99 475 уникальных пользователей, дубликатных значений их ID нет

In [13]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 0]['hid'].nunique()

38354

In [14]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 1]['hid'].nunique()

52491

In [15]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 2]['hid'].nunique()

8255

In [16]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 3]['hid'].nunique()

357

In [17]:
clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 4]['hid'].nunique()

18

Из них 38 354 не имеют зарегистрированного аккаунта нигде, 52 491 имеют аккаунт только в одной компании, 8 255 - в двух компаниях, 357 - в трёх и всего 18 пользователей во всех четырех
<br> Для наглядности - график ниже. Предполагаем, что пользователь на нём может входить в несколько компаний

In [226]:
values = [
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 0]['hid'].nunique(),
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 1]['hid'].nunique(),
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 2]['hid'].nunique(),
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 3]['hid'].nunique(),
    clean_df[clean_df[['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']].sum(axis=1) == 4]['hid'].nunique()
]
bin_names = ['Не зарегистрирован', 'В 1-ой компании', 'В 2-ух компаниях', 'В 3-х компаниях', 'В 4-х компаниях']

trace = go.Bar(x=bin_names, y=values, text=values)

layout = go.Layout(
    title='Кол-во зарегистрированных пользователей в компаниях',
    xaxis=dict(title='Кол-во компаний'),
    yaxis=dict(title='Кол-во пол-ей')
)

figure = go.Figure(data=trace, layout=layout)

figure.show()

Теперь можно посмотреть кол-во пользователей в каждой из компаний. 
<br> Тут также предполагаем, что пользователь может входить в несколько компаний

In [19]:
number_of_users = (clean_df[['hid','active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']]
 .groupby(['active_Red', 'has_Green_id', 'has_Black_id', 'has_Blue_id']).agg({'hid' : 'nunique'})
).reset_index()

In [20]:
renamed_columns = {
    'active_Red' : 'RC',
    'has_Green_id' : 'GC',
    'has_Black_id' : 'Black C',
    'has_Blue_id' : 'Blue C',
    'hid' : 'User ID'
}

In [21]:
number_of_users.rename(columns=renamed_columns, inplace=True)

In [22]:
number_of_users

,RC,GC,Black C,Blue C,User ID
0,0.0,0.0,0.0,0.0,38354
1,0.0,0.0,0.0,1.0,4819
2,0.0,0.0,1.0,0.0,331
3,0.0,0.0,1.0,1.0,29
4,0.0,1.0,0.0,0.0,390
5,0.0,1.0,0.0,1.0,40
6,0.0,1.0,1.0,0.0,50
7,0.0,1.0,1.0,1.0,3
8,1.0,0.0,0.0,0.0,46951
9,1.0,0.0,0.0,1.0,7528


In [23]:
number_of_users['Company'] = (
    number_of_users.apply(lambda row: ', '.join([col for col in number_of_users.columns if row[col] == 1]), axis=1)
)

In [24]:
number_of_users['Company'][0] = 'No Company'

C:\Users\60112813\AppData\Local\Temp\ipykernel_22120\4278020537.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
trace = go.Bar(x=number_of_users['Company'], y=number_of_users['User ID'])

layout = go.Layout(
    title='Кол-во зарегистрированных пользователей в компаниях',
    xaxis=dict(title='Принадлежность к компании'),
    yaxis=dict(title='Кол-во пол-ей')
)

figure = go.Figure(data=[trace], layout=layout)

figure.show()

В целом, что по сводной таблички, что по графику видно, что абсолютные лидеры по кол-ву пользователей это пользователи незарегистрированные нигде и пользователи из Red company.
<br> Далее идут пользователи, которые одновременно принадлежат и к Red company, и к Blue company, и пользователи, которые имееют аккаунт только в Blue company. (Для удобства отображение "Company" Было сокращено до "C")
<br> К сожалению, кол-во уникальных пользователей в каждой из компании не представляется возможным найти, так как нет даты регистрации в каждой из компаний, поэтому непонятно, в каком из сервисов пользователь зарегистрировался первее всего

### Распределение кол-ва успешных заказов в сервисе

К сожалению, датасет неполный и можно оценить кол-во успешных заказов и их доли только у компаний Green и Black
<br> Для большей точности я оставляю только тех пользователей, которые зарегистрированы хотя бы в одном из этих сервисов

In [26]:
orders_df = clean_df[['hid', 'Green_gvm_order_count', 'Black_gvm_order_count', 'has_Green_id', 'has_Black_id']]

In [27]:
orders_df = orders_df[(orders_df['Green_gvm_order_count'] + orders_df['Black_gvm_order_count'] > 0)]

In [28]:
orders_df[orders_df['has_Green_id'] > 0]['Green_gvm_order_count'].describe()

count    874.000000
mean       8.756293
std       15.872743
min        1.000000
25%        1.000000
50%        4.000000
75%        9.000000
max      168.000000
Name: Green_gvm_order_count, dtype: float64

In [29]:
orders_df[orders_df['has_Black_id'] > 0]['Black_gvm_order_count'].describe()

count    1065.000000
mean        4.051643
std         5.971751
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max        60.000000
Name: Black_gvm_order_count, dtype: float64

Всего успешных заказов у компании Green 7 653, максимальный заказ - 168 абстрактных штук, а у компании Black 4 315, 60 абстрактных штук
<br> Так как минимальное кол-во заказов в обоих компаниях равно одному, можно сказать, что в обоих компаниях каждый зарегистрированный пользователь заказывал хотя бы один раз, так что на долю успешных заказов в принципе в рамках компании смотреть бесполезно, она равно 100 %

### Распределение активных действий пол-тя в сервисе

К сожалению, опять же из-за неполноты датасеты, количество активных действий есть только у одной компании - Blue

In [30]:
activity_counts = clean_df[['hid', 'Blue_activity_count', 'has_Blue_id']]

In [31]:
activity_counts = activity_counts[activity_counts['has_Blue_id'] == 1]

In [32]:
activity_counts['Blue_activity_count'].describe()

count    12746.000000
mean         1.493802
std          5.484670
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max        237.000000
Name: Blue_activity_count, dtype: float64

Видим, что максимальное кол-во активных действий у пользователя 237, а минимальное 0. Тут долю активных/неактивных пол-ей посмотреть можно
<br> <b>P.S.</b> Также брал только тех пользователей, которые зареганы в Blue

In [33]:
(activity_counts[activity_counts['Blue_activity_count'] > 0]['hid'].nunique() / activity_counts['hid'].nunique()) * 100

27.875411893927506

Процентная доля активных пользователей составляет почти 28 % от общего кол-ва пользователей, зарегистрированных в сервисе
<br> Как будто сервис не очень популярный, если только около трети пользователей как-то в нем активничают

### Распределение GVM

Опять же, тут я оцениваю компании, если пользователь в ней зарегистрирован

In [34]:
gvm_total = (clean_df[[
    'hid', 'Blue_gmv_total', 
    'Green_gvm_total', 'Black_gvm_total', 
    'Blue_activity_count', 'Green_gvm_order_count', 
    'Black_gvm_order_count', 'has_Blue_id',
    'has_Black_id', 'has_Green_id'
    ]]
)

In [35]:
gvm_total_blue = gvm_total[gvm_total['has_Blue_id'] == 1]

In [36]:
gvm_total_blue.sum()

hid                      00003c7fa5e57ce672f1801f2679f5c40001aede5ec5c9...
Blue_gmv_total                                                    233704.0
Green_gvm_total                                                   874218.0
Black_gvm_total                                                  1990816.0
Blue_activity_count                                                19040.0
Green_gvm_order_count                                               1159.0
Black_gvm_order_count                                                989.0
has_Blue_id                                                        12746.0
has_Black_id                                                         265.0
has_Green_id                                                         155.0
dtype: object

Пользователи, зарегистрированные в компании Blue приносят тотальный GVM 233 704
<br> Однако, также они могут быть зарегистрированы в компаниях Black или Green или (возможно) в обоих.
<br> Компании Green они приносят 874 218 GVM, а компании Black больше всего - 1 990 816 GVM

In [37]:
gvm_total_green = gvm_total[gvm_total['has_Green_id'] == 1]

In [38]:
gvm_total_green.sum()

hid                      00096dc15d53c959cbff731eba795dfb00272111ca0bf4...
Blue_gmv_total                                                     15536.0
Green_gvm_total                                                  5935926.0
Black_gvm_total                                                  1556360.0
Blue_activity_count                                                  371.0
Green_gvm_order_count                                               7653.0
Black_gvm_order_count                                                698.0
has_Blue_id                                                          155.0
has_Black_id                                                         116.0
has_Green_id                                                         874.0
dtype: object

Пользователи зарегистрированные в компании Green приносят ей 5 935 926 GVM,
<br> Однако, они также могут быть зарегистрированы в двух других компаниях и приносят Blue 15 536 GVM и Black 1 556 360 GVM

In [39]:
gvm_total_black = gvm_total[gvm_total['has_Black_id'] == 1]

In [40]:
gvm_total_black.sum()

hid                      001c3f004c05017d60fa91f00cdbcdda003017284d22a5...
Blue_gmv_total                                                      6350.0
Green_gvm_total                                                  1187054.0
Black_gvm_total                                                  9354698.0
Blue_activity_count                                                  689.0
Green_gvm_order_count                                               1417.0
Black_gvm_order_count                                               4315.0
has_Blue_id                                                          265.0
has_Black_id                                                        1065.0
has_Green_id                                                         116.0
dtype: object

Пользователи, зарегистрированные в компании Black приносят ей 9 354 698 GVM,
Однако, ситуация такая же как и выше, поэтому Blue они также приносят 6 350 GVM, а Green 1 187 054 GVM

Для наглядности ниже построил график Кол-во пользователей - GVM Компании

In [41]:
gvm_total[gvm_total['has_Blue_id'] == 1]['hid'].nunique()

12746

In [42]:
data = [
        ['Blue', gvm_total[gvm_total['has_Blue_id'] == 1]['hid'].nunique(), gvm_total[gvm_total['has_Blue_id'] == 1]['Blue_gmv_total'].sum()],
        ['Green', gvm_total[gvm_total['has_Green_id'] == 1]['hid'].nunique(), gvm_total[gvm_total['has_Green_id'] == 1]['Green_gvm_total'].sum()],
        ['Black', gvm_total[gvm_total['has_Black_id'] == 1]['hid'].nunique(), gvm_total[gvm_total['has_Black_id'] == 1]['Black_gvm_total'].sum()]
]

In [43]:
rev_users = pd.DataFrame(data, columns=['Company name', 'Number of Users', 'GVM total'])

In [44]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Bar(x=rev_users['Company name'], y=rev_users['Number of Users'], name='Number of Users'), secondary_y=False)

# Add bar trace for revenue
fig.add_trace(go.Scatter(x=rev_users['Company name'], y=rev_users['GVM total'], name='GVM total', mode='markers'), secondary_y=True)

# Update layout
fig.update_layout(title='Number of Users and GVM total',
                  xaxis=dict(title='Company name'),
                  yaxis=dict(title='Number of Users'),
                  yaxis2=dict(title='GVM total', overlaying='y', side='right'))

fig.show()

Как видим, несмотря на то что компания Blue абсолютный лидер по кол-ву пользователей, самый большой GVM у компании Black

### Определние связей между GVM и активностей пользователей

Хочется оценить влияет ли на GVM некоторая активность, как, например, в случае с Blue. К сожалению, у остальных компаний есть только связки кол-ва успешных заказов и GVM, что оценивать не стоит, так как там скорее всего прямая зависимость

In [129]:
gvm_total_blue[['Blue_activity_count', 'Blue_gmv_total']].corr()

,Blue_activity_count,Blue_gmv_total
Blue_activity_count,1.000000,0.273668
Blue_gmv_total,0.273668,1.000000


In [124]:
graph_blue = gvm_total_blue[['hid','Blue_activity_count', 'Blue_gmv_total']].sample(100)

In [125]:
hid_to_replace = list(range(1, len(graph_blue) + 1))

In [126]:
graph_blue['hid'] = hid_to_replace

In [127]:
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=graph_blue['hid'], y=graph_blue['Blue_activity_count'], name='Кол-во активных действий'), secondary_y=False)

fig.add_trace(go.Scatter(x=graph_blue['hid'], y=graph_blue['Blue_gmv_total'], mode='lines', name='Общее GVM'), secondary_y=True)

fig.update_layout(
    title='Кол-во пользователей и общее GVM',
    xaxis_title='ID пользователя',
    yaxis=dict(title='Кол-во активных действий'),
    yaxis2=dict(title='Общее GVM', overlaying='y', side='right')
)

fig.show()

Как видим выше, численная корреляция не очень-то и большая. В целом это видно и из графика. Для удобства отображения я взял сэмпл из 100 пол-ей и заменил их ID из длинного на короткое число

Попробуем взять только тех пользователей, у которых была активность

In [50]:
blue_with_activity = (gvm_total_blue[
    [
    'hid', 'Blue_gmv_total', 'Blue_activity_count'
    ]
    ][
        gvm_total_blue[['hid', 'Blue_gmv_total', 'Blue_activity_count']]['Blue_activity_count'] > 1
    ]
)

In [51]:
blue_with_activity[['Blue_gmv_total', 'Blue_activity_count']].corr()

,Blue_gmv_total,Blue_activity_count
Blue_gmv_total,1.000000,0.272215
Blue_activity_count,0.272215,1.000000


In [52]:
graph_blue = blue_with_activity[['hid','Blue_activity_count', 'Blue_gmv_total']].sample(100)

In [53]:
hid_to_replace = list(range(1, len(graph_blue) + 1))

In [54]:
graph_blue['hid'] = hid_to_replace

In [55]:
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=graph_blue['hid'], y=graph_blue['Blue_activity_count'], name='Кол-во активных действий'), secondary_y=False)

fig.add_trace(go.Scatter(x=graph_blue['hid'], y=graph_blue['Blue_gmv_total'], mode='lines', name='Общее GVM'), secondary_y=True)

fig.update_layout(
    title='Кол-во пользователей и общее GVM',
    xaxis_title='ID пользователя',
    yaxis=dict(title='Кол-во активных действий'),
    yaxis2=dict(title='Общее GVM', overlaying='y', side='right')
)

fig.show()

Ситуация практически не изменилась, а корреляция стала даже чуть меньше

### Распределение геолокации и девайсов для использования

In [56]:
(clean_df[clean_df['is_msk'] == 1]['hid'].nunique() / clean_df['hid'].nunique()) * 100

11.893440562955517

Видим, что пользователей из Москвы всего около 12 %

In [57]:
clean_df[(clean_df[['Red_apple', 'Green_apple', 'Black_apple', 'Blue_apple']] > 0).any(axis=1)]['hid'].nunique()

10959

In [58]:
clean_df[(clean_df[['Red_android', 'Green_android', 'Black_android', 'Blue_android']] > 0).any(axis=1)]['hid'].nunique()

33073

In [59]:
clean_df[(clean_df[['Red_web', 'Green_web', 'Black_web', 'Blue_web']] > 0).any(axis=1)]['hid'].nunique()

67164

Распределение видим следующие: с яблоком 10 959 пользователей, с андроЕдом - 33 073, а с веб версией 67 164
<br> Опять же, не забываем, что у нескольких пользователей может быть несколько девайсов.

#### Разбиение пользователей на группы

In [60]:
red_users = clean_df[clean_df['active_Red'] == 1]

In [61]:
green_users = clean_df[clean_df['has_Green_id'] == 1]

In [62]:
blue_users = clean_df[clean_df['has_Blue_id'] == 1]

In [63]:
black_users = clean_df[clean_df['has_Black_id'] == 1]

#### Пользователи Red компании

In [64]:
red_users[red_users['Red_apple'] == 1]['hid'].nunique()

9013

In [65]:
(red_users[red_users['Red_apple'] == 1]['hid'].nunique() / red_users['hid'].nunique()) * 100

16.251645359635045

In [66]:
red_users[red_users['Red_android'] == 1]['hid'].nunique()

24855

In [67]:
(red_users[red_users['Red_android'] == 1]['hid'].nunique() / red_users['hid'].nunique()) * 100

44.81689175787519

In [68]:
red_users[red_users['Red_web'] == 1]['hid'].nunique()

45196

In [69]:
(red_users[red_users['Red_web'] == 1]['hid'].nunique() / red_users['hid'].nunique()) * 100

81.49443733208315

У пользователей Red компании разбиение видим следующее:
1. Эпл в меньшинстве, всего 9 013, 16 % общего кол-ва пользователей
2. АндроЕд наступает, 24 855, это почти 45 %
3. Веб-версия выигрывает, 45 196 человек, 81 %
<br> <b> P.S. </b> Да, процентов суммарно получается больше 100, так как один и тот же пользователь может пользоваться сервисом и с мобильного девайса (а может даже с двух, если богач), и с веб версии. Однако все равно можно сделать вывод, что веб версией пользуются больше всего

#### Пользователи Green компании

In [70]:
green_users[green_users['Green_apple'] == 1]['hid'].nunique()

226

In [71]:
(green_users[green_users['Green_apple'] == 1]['hid'].nunique() / green_users['hid'].nunique()) * 100

25.85812356979405

In [72]:
green_users[green_users['Green_android'] == 1]['hid'].nunique()

179

In [73]:
(green_users[green_users['Green_android'] == 1]['hid'].nunique() / green_users['hid'].nunique()) * 100

20.48054919908467

In [74]:
green_users[green_users['Green_web'] == 1]['hid'].nunique()

26

In [75]:
(green_users[green_users['Green_web'] == 1]['hid'].nunique() / green_users['hid'].nunique()) * 100

2.9748283752860414

In [76]:
green_users['hid'].nunique()

874

In [77]:
green_users[(green_users['Green_android'] == 0) & (green_users['Green_apple'] == 0) & (green_users['Green_web'] == 0)]['hid'].nunique()

470

In [78]:
(green_users[(green_users['Green_android'] == 0) & (green_users['Green_apple'] == 0) & (green_users['Green_web'] == 0)]['hid'].nunique() / green_users['hid'].nunique()) * 100

53.775743707093824

У зеленых ситуация следующая:
1. На удивление больше всего яблочников, 226 человек, это 25 %
2. АндроЕдов в этом случае поменьше, 179 человек, это 20 %
3. Веб версией почти никто не пользуется, всего 26 человек, это почти 3 %
4. Однако, есть большая прослойка людей, которая приносит GVM, но также не пользуется ни одним из девайсов, это почти 54 %, 470 человек. Так что делаю предположение, что возможно это какой-то в большей степени оффлайн сервис

#### Пользователи Blue компании

In [79]:
blue_users[blue_users['Blue_apple'] == 1]['hid'].nunique()

1674

In [80]:
(blue_users[blue_users['Blue_apple'] == 1]['hid'].nunique() / blue_users['hid'].nunique()) * 100

13.133532088498354

In [81]:
blue_users[blue_users['Blue_android'] == 1]['hid'].nunique()

2867

In [82]:
(blue_users[blue_users['Blue_android'] == 1]['hid'].nunique() / blue_users['hid'].nunique()) * 100

22.4933312411737

In [83]:
blue_users[blue_users['Blue_web'] == 1]['hid'].nunique()

2806

In [84]:
(blue_users[blue_users['Blue_web'] == 1]['hid'].nunique() / blue_users['hid'].nunique()) * 100

22.014749725404048

In [85]:
blue_users['hid'].nunique()

12746

In [86]:
blue_users[(blue_users['Blue_android'] == 0) & (blue_users['Blue_apple'] == 0) & (blue_users['Blue_web'] == 0)]['hid'].nunique()

6430

In [87]:
(blue_users[(blue_users['Blue_android'] == 0) & (blue_users['Blue_apple'] == 0) & (blue_users['Blue_web'] == 0)]['hid'].nunique() / blue_users['hid'].nunique()) * 100

50.447199121292954

У компании Blue примерно такая же ситуация как у Green:
1. Яблочников - 1674 человека, 13 %
2. АндроЕдов - 2867, почти 23 %
3. У веб версии - 2806 челвоека, 22 %
4. Также 6430 человек не пользуется ни одним из девайсов, это 50 %

#### Пользователи Black компании

In [88]:
black_users[black_users['Black_apple'] == 1]['hid'].nunique()

325

In [89]:
(black_users[black_users['Black_apple'] == 1]['hid'].nunique() / black_users['hid'].nunique()) * 100

30.51643192488263

In [90]:
black_users[black_users['Black_android'] == 1]['hid'].nunique()

229

In [91]:
(black_users[black_users['Black_android'] == 1]['hid'].nunique() / black_users['hid'].nunique()) * 100

21.502347417840376

In [92]:
black_users[black_users['Black_web'] == 1]['hid'].nunique()

171

In [93]:
(black_users[black_users['Black_web'] == 1]['hid'].nunique() / black_users['hid'].nunique()) * 100

16.056338028169016

In [94]:
black_users[(black_users['Black_android'] == 0) & (black_users['Black_apple'] == 0) & (black_users['Black_web'] == 0)]['hid'].nunique()

455

У Black компании ситуация примерно такая же

## Тестирование гипотез

В рамках этого тестового задания я решил выдвинуть несколько гипотез и проверить их:
1. Влияние проживания в Москве на GVM
2. Влияние девайса на GVM

Также можно было бы еще проверить:

1. Влияние региона на кол-во заказов (В случае с Black и Green)
2. Влияние девайса на кол-во заказов (Такие же компании)
3. Влияние девайсов на кол-во активных действий (В случае с Blue)

Но это осталось за рамками исследования

### Влияние проживания в Москве на GVM

Для успешного проведения t-test необходимо, чтобы все три предположения были верны, а именно:
1. Нормальность Выборки
2. Независимость, т.е. наблюдения в каждой выборке должны быть независимы
3. Примерно одинаковая дисперсия (либо отключать её в самом t-test)
Поэтому перед каждой проверкой гипотезы, я сначала буду проверять выборки на эти критерии

Для удобства и повышения уровня абстракции я поменяю названия некоторых колонок в заранее разделенных на компании датасетах, чтобы потом использовать их в функции

In [95]:
blue_users = blue_users[['hid', 'Blue_gmv_total', 'is_msk', 'Blue_apple', 'Blue_android', 'Blue_web']]

In [96]:
blue_users.rename(columns={'Blue_gmv_total' : 'gvm_total', 'Blue_apple' : 'apple', 'Blue_android' : 'android', 'Blue_web' : 'web'}, inplace=True)

In [97]:
green_users = green_users[['hid', 'Green_gvm_total', 'is_msk', 'Green_apple', 'Green_android', 'Green_web']]

In [98]:
green_users.rename(columns={'Green_gvm_total' : 'gvm_total', 'Green_apple' : 'apple', 'Green_android' : 'android', 'Green_web' : 'web'}, inplace=True)

In [99]:
black_users = black_users[['hid', 'Black_gvm_total', 'is_msk', 'Black_apple', 'Black_android', 'Black_web']]

In [100]:
black_users.rename(columns={'Black_gvm_total' : 'gvm_total', 'Black_apple' : 'apple', 'Black_android' : 'android', 'Black_web' : 'web'}, inplace=True)

In [101]:
def normality_check(list_of_df):
    from scipy.stats import shapiro 
    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue', 'green', 'black'])):
        print(f'Checking {df_name} dataset')
        ### Checking msk distribution
        if shapiro(df[df['is_msk'] == 1]['gvm_total'])[1] <= 0.05:
            print(f"The p-value of this data for msk region is {shapiro(df['gvm_total'])[1]}, the distribution is not normal")
        else:
            print('Check for normality for msk region is passed!')
        ### Checking non-msk distibution
        if shapiro(df[df['is_msk'] == 0]['gvm_total'])[1] <= 0.05:
            print(f"The p-value of this data for non-msk regions is {shapiro(df['gvm_total'])[1]}, the distribution is not normal")
        else:
            print('Check for normality for non-msk regions is passed!')
    

In [102]:
lst = [blue_users, green_users, black_users]

In [103]:
normality_check(lst)

Checking blue dataset
The p-value of this data for msk region is 0.0, the distribution is not normal
The p-value of this data for non-msk regions is 0.0, the distribution is not normal
Checking green dataset
The p-value of this data for msk region is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for non-msk regions is 1.2233335593555653e-42, the distribution is not normal
Checking black dataset
The p-value of this data for msk region is 0.0, the distribution is not normal
The p-value of this data for non-msk regions is 0.0, the distribution is not normal


c:\Users\60112813\Desktop\X5-homework\Python\X5_solution_venv\lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



Уже на первом тесте провал, так что t-test использовать нельзя
<br> Как альтернативу было решено попробовать <b>Mann Whitney U Test</b>, 
<br> которому пофиг на нормальность, но не пофиг на на независимость выборок

In [104]:
def independece_check(list_of_df):
    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue', 'green', 'black'])):
        print(f'Checking {df_name} dataset')
        ### Checking correlation
        if df[df['is_msk'] == 0]['gvm_total'].corr(df[df['is_msk'] == 1]['gvm_total']) > 0.5:
            print(f"The correlation is somewhat positive ({df[df['is_msk'] == 0].corr(df[df['is_msk'] == 1])}),observations are dependent to an extent")
        else:
            print(f"Observations are not dependent (correlation is {df[df['is_msk'] == 0]['gvm_total'].corr(df[df['is_msk'] == 1]['gvm_total'])})")
            print(f"Variance of non-msk is {df[df['is_msk'] == 0]['gvm_total'].var()}")
            print(f"Variance of msk is {df[df['is_msk'] == 1]['gvm_total'].var()}")

In [105]:
lst = [blue_users, green_users, black_users]

In [106]:
independece_check(lst)

Checking blue dataset
Observations are not dependent (correlation is nan)
Variance of non-msk is 131415.86695689775
Variance of msk is 88938.90869454492
Checking green dataset
Observations are not dependent (correlation is nan)
Variance of non-msk is 92189449.88330942
Variance of msk is 137351497.91600657
Checking black dataset
Observations are not dependent (correlation is nan)
Variance of non-msk is 171439977.71130556
Variance of msk is 218751217.08593073


Ни одна из выборок не является зависимой, хоть и корреляцию выдаёт nan. 
<br> Он может выдавать её в случае, если есть NaN в самой выборке, или же если в выборке нет дисперсии. 
<br> В других случаях корреляция настолько близка к нулю, что результат nan. 
<br> Дисперсия везде положительная, так что делаю выводы, что продолжать тест можно

Однако у теста Mann Whitney U Test есть предположение, что Коэффициент асимметрии (Skewness) должен быть похожим. Это я сейчас и проверю

In [107]:
def skewness_check(list_of_df):
    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue', 'green', 'black'])):
        print(f'Checking {df_name} dataset')
        ### Checking skewness
        if abs(df[df['is_msk'] == 0]['gvm_total'].skew() - df[df['is_msk'] == 1]['gvm_total'].skew()) < 1:
            print("The skewness of the two groups is similar.")
        else:
            print("The skewness of the two groups is different.")

In [108]:
lst = [blue_users, green_users, black_users]

In [109]:
skewness_check(lst)

Checking blue dataset
The skewness of the two groups is different.
Checking green dataset
The skewness of the two groups is different.
Checking black dataset
The skewness of the two groups is different.


К сожалению, результаты теста показывают, что и Mann Whitney U Test тут неприменим для тестирования гипотезы. (Так далеко я еще не заходил :D)
<br> Поэтому я попробую Permutation test
<br> Размер семплов делаю одинаковыми, опираясь на кол-во наблюдений в Мск регионе

In [115]:
def permutation_test(list_of_df):
    from mlxtend.evaluate import permutation_test

    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue', 'green', 'black'])):
        print(f'Checking {df_name} dataset')
        ### Permutation test
        len_msk_region = len(df[df['is_msk'] == 1])

        msk = df[df['is_msk'] == 1]['gvm_total']

        non_msk = df[df['is_msk'] == 1]['gvm_total'].sample(len_msk_region)

        p_value = permutation_test(msk, non_msk, method='approximate', num_rounds=1000, seed=0)

        print("Permutation Test p-value:", p_value)

In [116]:
lst = [blue_users, green_users, black_users]

In [117]:
permutation_test(lst)

Checking blue dataset
Permutation Test p-value: 1.0
Checking green dataset
Permutation Test p-value: 1.0
Checking black dataset
Permutation Test p-value: 1.0


Welp  ¯\ _ (ツ)_/¯
<br> Зависимости GVM от региона нет от слова совсем. Хотя я все равно бы чуть аккуратнее относился к результатам теста

<b> График для затравочки </br>

In [ ]:
data = [
        ['Blue', gvm_total[gvm_total['has_Blue_id'] == 1]['hid'].nunique(), gvm_total[gvm_total['has_Blue_id'] == 1]['Blue_gmv_total'].sum()],
        ['Green', gvm_total[gvm_total['has_Green_id'] == 1]['hid'].nunique(), gvm_total[gvm_total['has_Green_id'] == 1]['Green_gvm_total'].sum()],
        ['Black', gvm_total[gvm_total['has_Black_id'] == 1]['hid'].nunique(), gvm_total[gvm_total['has_Black_id'] == 1]['Black_gvm_total'].sum()]
]

In [130]:
data = [
    ['Blue_msk_users', blue_users[blue_users['is_msk'] == 1]['hid'].nunique(), blue_users[blue_users['is_msk'] == 1]['gvm_total'].sum()],
    ['Blue_non_msk_users', blue_users[blue_users['is_msk'] == 0]['hid'].nunique(), blue_users[blue_users['is_msk'] == 0]['gvm_total'].sum()],
    ['Green_msk_users', green_users[green_users['is_msk'] == 1]['hid'].nunique(), green_users[green_users['is_msk'] == 1]['gvm_total'].sum()],
    ['Green_non_msk_users', green_users[green_users['is_msk'] == 0]['hid'].nunique(), green_users[green_users['is_msk'] == 0]['gvm_total'].sum()],
    ['Black_msk_users', black_users[black_users['is_msk'] == 1]['hid'].nunique(), black_users[black_users['is_msk'] == 1]['gvm_total'].sum()],
    ['Black_non_msk_users', black_users[black_users['is_msk'] == 0]['hid'].nunique(), black_users[black_users['is_msk'] == 0]['gvm_total'].sum()]
    
]

In [131]:
comp_users = pd.DataFrame(data, columns=['Company name', 'Number of Users', 'GVM total'])

In [134]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(go.Bar(x=comp_users['Company name'], y=comp_users['Number of Users'], name='Number of Users'), secondary_y=False)

# Add bar trace for revenue
fig.add_trace(go.Scatter(x=comp_users['Company name'], y=comp_users['GVM total'], name='GVM total', mode='markers'), secondary_y=True)

# Update layout
fig.update_layout(title='Number of Users and GVM total',
                  xaxis=dict(title='Company name'),
                  yaxis=dict(title='Number of Users'),
                  yaxis2=dict(title='GVM total', overlaying='y', side='right'))

fig.show()

Визуально зависимости не видно. Только в случае с Green как будто что-то есть, но там в Москве намного больше пользователей напорядок больше, так что это могло сыграть свою роль в Total GVM

### Влияние девайса на GVM

Тут всё то же самое, сначала проверяю все наблюдения подходят ли они под тот или иной тест

In [144]:
def normality_check(list_of_df):
    from scipy.stats import shapiro 
    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue', 'green', 'black'])):
        print(f'Checking {df_name} dataset')
        ### Checking apple
        if shapiro(df[df['apple'] == 1]['gvm_total'])[1] <= 0.05:
            print(f"The p-value of this data for apple is {shapiro(df['gvm_total'])[1]}, the distribution is not normal")
        else:
            print('Check for normality for apple is passed!')
        ### Checking apple
        if shapiro(df[df['apple'] == 0]['gvm_total'])[1] <= 0.05:
            print(f"The p-value of this data for apple is {shapiro(df['gvm_total'])[1]}, the distribution is not normal")
        else:
            print('Check for normality for apple is passed!')
        
        ### Checking android
        if shapiro(df[df['android'] == 1]['gvm_total'])[1] <= 0.05:
            print(f"The p-value of this data for android is {shapiro(df['gvm_total'])[1]}, the distribution is not normal")
        else:
            print('Check for normality for android is passed!')
        ### Checking android
        if shapiro(df[df['android'] == 0]['gvm_total'])[1] <= 0.05:
            print(f"The p-value of this data for android is {shapiro(df['gvm_total'])[1]}, the distribution is not normal")
        else:
            print('Check for normality for android is passed!')
    
        ### Checking web
        if shapiro(df[df['web'] == 1]['gvm_total'])[1] <= 0.05:
            print(f"The p-value of this data for web is {shapiro(df['gvm_total'])[1]}, the distribution is not normal")
        else:
            print('Check for normality for web is passed!')
        ### Checking web
        if shapiro(df[df['web'] == 0]['gvm_total'])[1] <= 0.05:
            print(f"The p-value of this data for web is {shapiro(df['gvm_total'])[1]}, the distribution is not normal")
        else:
            print('Check for normality for web is passed!')

In [145]:
lst = [blue_users, green_users, black_users]

In [146]:
normality_check(lst)

Checking blue dataset
The p-value of this data for apple is 0.0, the distribution is not normal
The p-value of this data for apple is 0.0, the distribution is not normal
The p-value of this data for android is 0.0, the distribution is not normal
The p-value of this data for android is 0.0, the distribution is not normal
The p-value of this data for web is 0.0, the distribution is not normal
The p-value of this data for web is 0.0, the distribution is not normal
Checking green dataset
The p-value of this data for apple is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for apple is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for android is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for android is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for web is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for web is 1.223333

c:\Users\60112813\Desktop\X5-homework\Python\X5_solution_venv\lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



Как видно из тестов, ни одно из распределений не является нормальным.
<br> Можно было бы сузить выборку до тех пользователей, которые имеют хотя бы какой-то из девайсов, но учитывая факты, которые нашлись ранее - нельзя.
<br> В случае всех трех компаний были найдены люди, которые не пользуются ни одним из девайсов совсем

In [155]:
black_users[(black_users['android'] == 0) & (black_users['apple'] == 0) & (black_users['web'] == 0)]['gvm_total'].sum()

3667294.0

In [156]:
blue_users[(blue_users['android'] == 0) & (blue_users['apple'] == 0) & (blue_users['web'] == 0)]['gvm_total'].sum()

0.0

In [158]:
green_users[(green_users['android'] == 0) & (green_users['apple'] == 0) & (green_users['web'] == 0)]['gvm_total'].sum()

3008234.0

Здесь мы видим, что пользователи без девайсов приносят Black и Green довольно много, так что выборку можно уменьшить только у Blue, хотя я не уверен, что это на что-то повлияет

In [159]:
new_blue = blue_users[(blue_users['android'] == 1) | (blue_users['apple'] == 1) | (blue_users['web'] == 1)]

In [161]:
new_blue.head()

,hid,gvm_total,is_msk,apple,android,web
28,00024b3bdd5d565a4338f76f5b8f68f3,0.0,0.0,1.0,0.0,0.0
51,0003b76af76edd4bc65cd6e0dd78b042,0.0,0.0,0.0,1.0,0.0
59,0004324c7b8f97c9daec7916c3878cf6,0.0,0.0,0.0,1.0,0.0
72,00055e7fe62b84221d156dd46bc9f9c3,0.0,0.0,0.0,1.0,0.0
80,0005c61e7f6c89d0eb56323570ae446a,0.0,0.0,0.0,0.0,1.0


In [167]:
lst = [new_blue, green_users, black_users]

In [168]:
normality_check(lst)

Checking blue dataset
The p-value of this data for apple is 0.0, the distribution is not normal
The p-value of this data for apple is 0.0, the distribution is not normal
The p-value of this data for android is 0.0, the distribution is not normal
The p-value of this data for android is 0.0, the distribution is not normal
The p-value of this data for web is 0.0, the distribution is not normal
The p-value of this data for web is 0.0, the distribution is not normal
Checking green dataset
The p-value of this data for apple is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for apple is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for android is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for android is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for web is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for web is 1.223333

c:\Users\60112813\Desktop\X5-homework\Python\X5_solution_venv\lib\site-packages\scipy\stats\_morestats.py:1882: UserWarning:

p-value may not be accurate for N > 5000.



<b> Ничего не изменилось </b>
<br> Попробуем взять семпл у blue

In [169]:
lst = [new_blue.sample(1000), green_users, black_users]
normality_check(lst)

Checking blue dataset
The p-value of this data for apple is 0.0, the distribution is not normal
The p-value of this data for apple is 0.0, the distribution is not normal
The p-value of this data for android is 0.0, the distribution is not normal
The p-value of this data for android is 0.0, the distribution is not normal
The p-value of this data for web is 0.0, the distribution is not normal
The p-value of this data for web is 0.0, the distribution is not normal
Checking green dataset
The p-value of this data for apple is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for apple is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for android is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for android is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for web is 1.2233335593555653e-42, the distribution is not normal
The p-value of this data for web is 1.223333

<b> Всё также, без изменений, переходим к проверкам Mann Whitney U Test <b>

In [171]:
def independece_check(list_of_df):
    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue', 'green', 'black'])):
        print(f'Checking {df_name} dataset')
        ### Checking apple correlation
        if df[df['apple'] == 0]['gvm_total'].corr(df[df['apple'] == 1]['gvm_total']) > 0.5:
            print(f"The correlation for apple is somewhat positive ({df[df['apple'] == 0].corr(df[df['apple'] == 1])}),observations are dependent to an extent")
        else:
            print(f"Observations are not dependent (correlation is {df[df['apple'] == 0]['gvm_total'].corr(df[df['apple'] == 1]['gvm_total'])})")
            print(f"Variance of not apple is {df[df['apple'] == 0]['gvm_total'].var()}")
            print(f"Variance of apple is {df[df['apple'] == 1]['gvm_total'].var()}")

        ### Checking android correlation
        if df[df['android'] == 0]['gvm_total'].corr(df[df['android'] == 1]['gvm_total']) > 0.5:
            print(f"The correlation for android is somewhat positive ({df[df['androi'] == 0].corr(df[df['android'] == 1])}),observations are dependent to an extent")
        else:
            print(f"Observations are not dependent (correlation is {df[df['android'] == 0]['gvm_total'].corr(df[df['android'] == 1]['gvm_total'])})")
            print(f"Variance of not android is {df[df['android'] == 0]['gvm_total'].var()}")
            print(f"Variance of android is {df[df['android'] == 1]['gvm_total'].var()}")

        ### Checking web correlation
        if df[df['web'] == 0]['gvm_total'].corr(df[df['web'] == 1]['gvm_total']) > 0.5:
            print(f"The correlation for web is somewhat positive ({df[df['web'] == 0].corr(df[df['web'] == 1])}),observations are dependent to an extent")
        else:
            print(f"Observations are not dependent (correlation is {df[df['web'] == 0]['gvm_total'].corr(df[df['web'] == 1]['gvm_total'])})")
            print(f"Variance of not web is {df[df['web'] == 0]['gvm_total'].var()}")
            print(f"Variance of web is {df[df['web'] == 1]['gvm_total'].var()}")

In [172]:
lst = [blue_users, green_users, black_users]

In [173]:
independece_check(lst)

Checking blue dataset
Observations are not dependent (correlation is nan)
Variance of not apple is 36810.96861692346
Variance of apple is 70224.06963217193
Observations are not dependent (correlation is nan)
Variance of not android is 45952.96037117589
Variance of android is 45351.63600233764
Observations are not dependent (correlation is nan)
Variance of not web is 45813.08731101706
Variance of web is 45513.54069850322
Checking green dataset
Observations are not dependent (correlation is nan)
Variance of not apple is 126951534.19424887
Variance of apple is 126890375.19897737
Observations are not dependent (correlation is nan)
Variance of not android is 123590394.9964547
Variance of android is 141252886.83221397
Observations are not dependent (correlation is nan)
Variance of not web is 127885342.50909424
Variance of web is 104062689.05846155
Checking black dataset
Observations are not dependent (correlation is nan)
Variance of not apple is 175933893.18433237
Variance of apple is 230237

Круто, всё как в прошлом тестировании, теперь надо проверить skewness, чтобы окончательно решить, подходит ли нам тест Mann Whitney U Test

In [174]:
def skewness_check(list_of_df):
    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue', 'green', 'black'])):
        print(f'Checking {df_name} dataset')
        ### Checking apple skewness
        if abs(df[df['apple'] == 0]['gvm_total'].skew() - df[df['apple'] == 1]['gvm_total'].skew()) < 1:
            print("The skewness of the two groups is similar.")
        else:
            print("The skewness of the two groups is different.")

        ### Checking android skewness
        if abs(df[df['android'] == 0]['gvm_total'].skew() - df[df['android'] == 1]['gvm_total'].skew()) < 1:
            print("The skewness of the two groups is similar.")
        else:
            print("The skewness of the two groups is different.")

        ### Checking web skewness
        if abs(df[df['web'] == 0]['gvm_total'].skew() - df[df['web'] == 1]['gvm_total'].skew()) < 1:
            print("The skewness of the two groups is similar.")
        else:
            print("The skewness of the two groups is different.")

In [175]:
lst = [blue_users, green_users, black_users]

In [176]:
skewness_check(lst)

Checking blue dataset
The skewness of the two groups is different.
The skewness of the two groups is different.
The skewness of the two groups is different.
Checking green dataset
The skewness of the two groups is different.
The skewness of the two groups is similar.
The skewness of the two groups is similar.
Checking black dataset
The skewness of the two groups is similar.
The skewness of the two groups is different.
The skewness of the two groups is different.


Итак, результаты следующие:
1. Для Blue Mann Whitney U Test не подходит никак
2. Для Green он подходит только в случае Андроида и Веба, в случае с Apple буду использовать подход из тестирования прошлой гипотезы
3. Для Black он подходит наоборот только для Apple, а для АндроЕда и веба буду также использовать подход из прошлой гипотезы

#### Blue company testing

Определяем размер семплов

In [181]:
(blue_users['apple'] == 1).sum()

1674

In [182]:
(blue_users['apple'] == 0).sum()

4642

In [185]:
(blue_users['android'] == 1).sum()

2867

In [186]:
(blue_users['android'] == 0).sum()

3449

In [187]:
(blue_users['web'] == 1).sum()

2806

In [188]:
(blue_users['web'] == 0).sum()

3510

In [200]:
def permutation_test(list_of_df):
    from mlxtend.evaluate import permutation_test

    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue'])):
        print(f'Checking {df_name} dataset')
        ### Permutation test apple
        len_apple = len(df[df['apple'] == 1])

        apple = df[df['apple'] == 1]['gvm_total']

        non_apple = df[df['apple'] == 1]['gvm_total'].sample(len_apple)

        p_value = permutation_test(apple, non_apple, method='approximate', num_rounds=1000, seed=0)

        print("Permutation Test apple p-value:", p_value)

        ### Permutation test android
        len_android = len(df[df['android'] == 1])

        android = df[df['android'] == 1]['gvm_total']

        non_android = df[df['android'] == 1]['gvm_total'].sample(len_android)

        p_value = permutation_test(android, non_android, method='approximate', num_rounds=1000, seed=0)

        print("Permutation Test android p-value:", p_value)

        ### Permutation test web
        len_web = len(df[df['web'] == 1])

        web = df[df['web'] == 1]['gvm_total']

        non_web = df[df['web'] == 1]['gvm_total'].sample(len_web)

        p_value = permutation_test(web, non_web, method='approximate', num_rounds=1000, seed=0)

        print("Permutation Test web p-value:", p_value)

In [201]:
lst = [blue_users]

In [202]:
permutation_test(lst)

Checking blue dataset
Permutation Test apple p-value: 1.0
Permutation Test android p-value: 1.0
Permutation Test web p-value: 1.0


Результаты показывают что в случае Blue влияния девайса на GVM нет

#### Green company testing

In [203]:
def permutation_test(list_of_df):
    from mlxtend.evaluate import permutation_test

    for idx, (df, df_name) in enumerate(zip(list_of_df, ['blue'])):
        print(f'Checking {df_name} dataset')
        ### Permutation test apple
        len_apple = len(df[df['apple'] == 1])

        apple = df[df['apple'] == 1]['gvm_total']

        non_apple = df[df['apple'] == 1]['gvm_total'].sample(len_apple)

        p_value = permutation_test(apple, non_apple, method='approximate', num_rounds=1000, seed=0)

        print("Permutation Test apple p-value:", p_value)

In [204]:
lst = [green_users]

In [205]:
permutation_test(lst)

Checking blue dataset
Permutation Test apple p-value: 1.0


Опять же, у Apple и GVM взаимосвязей нет, проверим андроЕда и веб

In [214]:
def mann_test(list_of_df):
    import scipy.stats as stats

    for idx, (df, df_name) in enumerate(zip(list_of_df, ['green'])):
        print(f'Checking {df_name} dataset')
        ### Mann test Android
        len_android = len(df[df['android'] == 1])

        group_android = df[df['android'] == 1]['gvm_total']
        group_non_android = df[df['android'] == 0]['gvm_total'].sample(len_android)

        t_statistic, p_value_t = stats.ttest_ind(group_android, group_non_android)

        u_statistic, p_value_u = stats.mannwhitneyu(group_android, group_non_android, alternative='two-sided')

        print("Android Independent Samples t-test:")
        print("Android t-statistic:", t_statistic)
        print("Android p-value:", p_value_t)

        print("\nAndroid Mann-Whitney U test:")
        print("Android U-statistic:", u_statistic)
        print("Android p-value:", p_value_u)

        ### Mann test Web
        len_web = len(df[df['web'] == 1])

        group_web = df[df['web'] == 1]['gvm_total']
        group_non_web = df[df['web'] == 0]['gvm_total'].sample(len_web)

        t_statistic, p_value_t = stats.ttest_ind(group_web, group_non_web)

        u_statistic, p_value_u = stats.mannwhitneyu(group_web, group_non_web, alternative='two-sided')

        print("\nWeb Independent Samples t-test:")
        print("Web t-statistic:", t_statistic)
        print("Web p-value:", p_value_t)

        print("\nWeb Mann-Whitney U test:")
        print("Web U-statistic:", u_statistic)
        print("Web p-value:", p_value_u)

In [215]:
lst = [green_users]

In [216]:
mann_test(lst)

Checking green dataset
Android Independent Samples t-test:
Android t-statistic: -0.2648950615207005
Android p-value: 0.791243608444113

Android Mann-Whitney U test:
Android U-statistic: 15571.0
Android p-value: 0.6465008326342406

Web Independent Samples t-test:
Web t-statistic: -0.516045292314959
Web p-value: 0.6080974604671063

Web Mann-Whitney U test:
Web U-statistic: 340.5
Web p-value: 0.9707933203865761


Всё также, ни в одном из девайсов связи с GVM не выявлено

#### Black company

In [218]:
def mann_test(list_of_df):
    import scipy.stats as stats

    for idx, (df, df_name) in enumerate(zip(list_of_df, ['black'])):
        print(f'Checking {df_name} dataset')
        ### Mann test Apple
        len_apple = len(df[df['apple'] == 1])

        group_apple = df[df['apple'] == 1]['gvm_total']
        group_non_apple = df[df['apple'] == 0]['gvm_total'].sample(len_apple)

        t_statistic, p_value_t = stats.ttest_ind(group_apple, group_non_apple)

        u_statistic, p_value_u = stats.mannwhitneyu(group_apple, group_non_apple, alternative='two-sided')

        print("Apple Independent Samples t-test:")
        print("Apple t-statistic:", t_statistic)
        print("Apple p-value:", p_value_t)

        print("\nApple Mann-Whitney U test:")
        print("Apple U-statistic:", u_statistic)
        print("Apple p-value:", p_value_u)

In [ ]:
lst = [black_users]

In [219]:
mann_test(lst)

Checking black dataset
Apple Independent Samples t-test:
Apple t-statistic: 2.066606801190467
Apple p-value: 0.039342723145958346

Apple Mann-Whitney U test:
Apple U-statistic: 29358.0
Apple p-value: 0.005946031238549428


К сожалению, насколько я понимаю, прошлая проверка независимости дала сбой, либо я где-то ошибся, либо просто проверки по корреляции недостаточно.
<br> Также, к сожалению, про такую проверку на независимость вспомнил только сейчас.
<br> Так как p-value независимости выборок < 0.05, то результаты тесты воспринимать нельзя, так как нарушается одно из предположенией теста.
<br> Попробую прогнать permutation test на всех девайсах :/

In [220]:
def permutation_test(list_of_df):
    from mlxtend.evaluate import permutation_test

    for idx, (df, df_name) in enumerate(zip(list_of_df, ['black'])):
        print(f'Checking {df_name} dataset')
        ### Permutation test apple
        len_apple = len(df[df['apple'] == 1])

        apple = df[df['apple'] == 1]['gvm_total']

        non_apple = df[df['apple'] == 1]['gvm_total'].sample(len_apple)

        p_value = permutation_test(apple, non_apple, method='approximate', num_rounds=1000, seed=0)

        print("Permutation Test apple p-value:", p_value)

        ### Permutation test android
        len_android = len(df[df['android'] == 1])

        android = df[df['android'] == 1]['gvm_total']

        non_android = df[df['android'] == 1]['gvm_total'].sample(len_android)

        p_value = permutation_test(android, non_android, method='approximate', num_rounds=1000, seed=0)

        print("Permutation Test android p-value:", p_value)

        ### Permutation test web
        len_web = len(df[df['web'] == 1])

        web = df[df['web'] == 1]['gvm_total']

        non_web = df[df['web'] == 1]['gvm_total'].sample(len_web)

        p_value = permutation_test(web, non_web, method='approximate', num_rounds=1000, seed=0)

        print("Permutation Test web p-value:", p_value)

In [221]:
lst = [black_users]

In [222]:
permutation_test(lst)

Checking black dataset
Permutation Test apple p-value: 1.0
Permutation Test android p-value: 1.0
Permutation Test web p-value: 1.0


Получается тестирование обеих гипотез не выявило никаких взаимосвязей, хотя Permutation test я ранее не использовал, поэтому к результатам исследования отнесся бы немного скептически.

<br> Ну вот и всё, спасибо за уделённое время для проверки тестового!